In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import openpyxl
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo

# Comando para ignorar os UserWarning dados pelo Pyhton
import warnings
warnings.filterwarnings('ignore')

def readSheets (file,sheetNames=[]):
    print(f"Abrindo o arquivo {file}")
    print(f"Esse processo pode demorar 1 ou 2 minutos...")
    if len(sheetNames)==0:
        print(f"Lendo as planilhas do arquivo {file}")
        dataFrames = pd.read_excel(file,sheet_name=None)
    else:
        print(f"Lendo as planilhas {sheetNames} do arquivo {file}")
        dataFrames = {sheetName:pd.read_excel(file,sheetName) for sheetName in sheetNames}
    print("O arquivo foi importado com sucesso.")
    return dataFrames

file = "leituras-janeiro2002-junho2024.xlsx"
registrationsSheet = "Cadastro Junho2024"
measuresSheet = "Resumo"
df_global = readSheets(file=file,sheetNames=[registrationsSheet,measuresSheet])


Abrindo o arquivo leituras-janeiro2002-junho2024.xlsx
Esse processo pode demorar 1 ou 2 minutos...
Lendo as planilhas ['Cadastro Junho2024', 'Resumo'] do arquivo leituras-janeiro2002-junho2024.xlsx
O arquivo foi importado com sucesso.


In [2]:
cadastro = df_global[registrationsSheet].copy(deep=True)
leituras = df_global[measuresSheet].copy(deep=True)
columnsRegistrations = list(cadastro.columns)
columnsMeasures = list(leituras.columns)

# Nome das colunas do cadastro
colunaCodigoCadastro = "Código"
colunaCotaFundo = "Cota do Fundo (m)"
colunaCotaTopo = "Cota do Topo (m)"
colunaAtencao = "Nível de Atenção (Maior que)"
colunaAlerta = "Nível de Alerta (Maior que)"
colunaEmergencia = "Nível de Emergência (Maior que)"
colunaEstrutura = "Estrutura Geotécnica"
colunaTipo = "Tipo de Instrumento"

# Nome das colunas da medição
colunaCodigo = "Código do Instrumento"
colunaSituacaoMedicao = "Situação da Medição"
colunaOutlier = "Outlier"
columnValues = "Valor Final"
columnIndex = "Data de Medição"
columnColumns = "Código do Instrumento"
columnGrandeza = "Grandeza física"
columnUnidade = "Unidade de Medida"


dataFimDefault = pd.Timestamp('2024-07-01')

# Verificação de Cadastro
estruturasInteresse = ['ED Monjolo',
                       'ED Vale das Cobras',
                       'Porteirinha',
                       'Diogo',
                       'Monjolo',
                       'Elefante',
                       'Pluviometria Água Limpa']
for i in list(leituras[~(leituras["Cota de Fundo (m)"].notna())]["Estrutura"].drop_duplicates()):
    if i in estruturasInteresse:
        raise Exception(f"Há um instrumento na estrutura {i} que não está no cadastro.")

In [3]:
def analisarLeiturasTipoInstrumento(tipoinstrumento,
                                    listColumns:list = [columnGrandeza,columnUnidade,colunaCodigo,"Estrutura"]
                                    ):
    return pd.concat([leituras[leituras[colunaTipo] == tipoinstrumento]])[listColumns].drop_duplicates().dropna()

def get_info_instrument(nameInstrument,nameInfo=colunaCodigoCadastro,df=cadastro):
    return df[df[colunaCodigoCadastro]==nameInstrument][nameInfo].values[0]

def tratarDados(nomeInstrumento:str="AGLPL001",
                df:pd.DataFrame=leituras,
                dataInicio:pd.Timestamp|None=None,
                dataFim:pd.Timestamp|None=None,
                seco:bool=False
                ):
    
    leiturasFiltradas = df.copy(deep=True)
    leiturasFiltradas = leiturasFiltradas[leiturasFiltradas[colunaCodigo]==nomeInstrumento][leiturasFiltradas[colunaSituacaoMedicao]=="Realizada"][leiturasFiltradas[colunaOutlier]!="Sim"]
    if seco:
        leiturasFiltradas = leiturasFiltradas[leiturasFiltradas["Condição Adversa"]=="SECO"]
    
    leiturasFiltradas = leiturasFiltradas[leiturasFiltradas["Condição Adversa"]!="JORRANTE"]

    pivotLeiturasFiltradas = leiturasFiltradas.pivot_table(values=columnValues,
                                                           index=columnIndex,
                                                           columns=columnColumns,
                                                           dropna=True,
                                                           aggfunc=max
                                                           )
    if dataInicio!=None:
        pivotLeiturasFiltradas = pivotLeiturasFiltradas.loc[dataInicio:]
    if dataFim!=None:
        pivotLeiturasFiltradas = pivotLeiturasFiltradas.loc[:dataFim]

    if seco:
        try:
            pivotLeiturasFiltradas.rename(columns={nomeInstrumento:f"{nomeInstrumento} (seco)"},inplace=True)
        except Exception:
            return pivotLeiturasFiltradas
    return pivotLeiturasFiltradas

def definirLimites(nomeInstrumento:str="AGLPL001", df:pd.DataFrame=leituras):
    serie = tratarDados(nomeInstrumento=nomeInstrumento,df=df).dropna()
    if len(serie)!=0:
        return pd.DataFrame(dict(
            data=dict(
                min=serie.sort_index().index[0],
                max=serie.sort_index().index[-1]
            ),
            leitura=dict(
                min=serie.sort_values(by=nomeInstrumento).values[0][0],
                max=serie.sort_values(by=nomeInstrumento).values[-1][0]
            )
        ))
    else:
        return pd.DataFrame(dict(
            data=dict(
                min=pd.Timestamp(0),
                max=pd.Timestamp(0)
            ),
            leitura=dict(
                min=0,
                max=0
            )
        ))

def listarInstrumentos(estrutura="",tipoInstrumento="",df=cadastro):
    if estrutura!="":
        df = df[df[colunaEstrutura]==estrutura]
    if tipoInstrumento!="":
        df = df[df[colunaTipo]==tipoInstrumento]
    return list(df["Código"].values)

def exibirLimites(estrutura,tipoInstrumento):
    pzs = listarInstrumentos(estrutura=estrutura,tipoInstrumento=tipoInstrumento)
    for pz in pzs:
        print(pz)
        print(definirLimites(pz))
        print()

def gerarHistorico(nomeInstrumento,
                   nomeInstrumentoPluv="AGLPL001",
                   dataFim=dataFimDefault,
                   temNVControle=True,
                   temCotaFundo=True,
                   temCotaTopo=True,
                   temPLV=True
                   ):
    print(1)
    pluv = tratarDados(nomeInstrumentoPluv)
    inst = tratarDados(nomeInstrumento)
    seco = tratarDados(nomeInstrumento,seco=True)
    historico = pd.concat([pluv,
                           inst,
                           seco],join="outer",axis=1)
    seriesExtras = [
        [temNVControle,"atenção",colunaAtencao],
        [temNVControle,"alerta",colunaAlerta],
        [temNVControle,"emergência",colunaEmergencia],
        [temCotaFundo,"cota de fundo",colunaCotaTopo],
        [temCotaTopo,"cota de topo",colunaCotaFundo]
    ]
    for boolean,string,column in seriesExtras:
        if boolean:
            historico.loc[:,f"{nomeInstrumento} ({string})"] = get_info_instrument(nomeInstrumento,column)
    return historico

def intervaloPerfeitoData(nomeInstrumento):
    
    min = definirLimites(nomeInstrumento)["data"]["min"]
    if min.month<=6:
        minCorrigido = pd.Timestamp(day=1,month=1,year=min.year)
    elif min.month==7 and min.day==1:
        minCorrigido = min        
    else:
        minCorrigido = pd.Timestamp(day=1,month=7,year=min.year)
    
    max = definirLimites(nomeInstrumento)["data"]["max"]
    if max.month<=6:
        maxCorrigido = pd.Timestamp(day=1,month=7,year=max.year)
    elif max.month==7 and max.day==1:
        maxCorrigido = max
    else:
        maxCorrigido = pd.Timestamp(day=1,month=1,year=max.year+1)
    
    return minCorrigido,maxCorrigido

def intervaloPerfeito(nomeInstrumento,dV:float|int|None=None,superior=True,min=None,max=None):
    if min==None or max==None:
        min = definirLimites(nomeInstrumento)["leitura"]["min"]
        max = definirLimites(nomeInstrumento)["leitura"]["max"]
    
    if max<min:
        raise Exception("Valor max menor do que valor min.")
    if dV==None:
        dVDefault = [
            0.01,
            0.02,
            0.05,
            0.1,
            0.2,
            0.5,
            1,
            2,
            5,
            10,
            20,
            50]
        L = abs(max-min)
        for dVi in sorted(dVDefault):
            if 0<L/dVi<=10:
                dV = dVi
                break
            else:
                dV=1
    if superior:
        minPerf, maxPerf = (min//dV)*dV-dV,(max//dV)*dV+dV
        if minPerf<0:
            minPerf=0
        return minPerf, maxPerf, dV
    else:
        raise Exception("Ainda não foi implementado o superior=False")

def gerarGrafico(nomeInstrumento,
                 nomeInstrumentoPluv="AGLPL001",
                 dataFim=dataFimDefault,
                 temNVControle=False,
                 temCotaFundo=False,
                 temCotaTopo=False,
                 temPLV=True,
                 intervaloY = [],
                 ymax_outlier = None,
                 ymin_outlier = None
                 ):
    
    # Visitar para ver estilos de marcadores: https://plotly.com/python/marker-style/
    
    dataInicio,dataFim = intervaloPerfeitoData(nomeInstrumento)
    # historico = gerarHistorico(nomeInstrumento)[dataInicio:dataFim]
    historico = gerarHistorico(nomeInstrumento)

    df_ = historico.copy(deep=True)
    
    estrutura = get_info_instrument(nomeInstrumento,colunaEstrutura)
    tipo = get_info_instrument(nomeInstrumento,colunaTipo)
    instrumentoToEixo = {
        "Régua de Reservatório":"Elevação (m)",
        "Medidor de Nível de Água":"Elevação (m)",
        "Piezômetro":"Elevação (m)",
        
        "Medidor de Vazão":"Vazão (L/s)",
        
        "Marco Topográfico":"Vetor Deslocamento (m)",
        "Radar Orbital":"Vetor Deslocamento (m)",
        "Estação Topográfica":"Vetor Deslocamento (m)",
        "Radar":"Vetor Deslocamento (m)",
        
        "Pluviômetro":"Pluviometria (mm)",
        "Pluviógrafo":"Precipitação (mm)",
        
        "Câmera":"",
        "Poço":"",
        }
    nomeEixo = instrumentoToEixo[tipo]

    pluvLimInf,pluvLimSup,dV_pluv = intervaloPerfeito(nomeInstrumentoPluv)
    data = []

    if temPLV:
        y_bar = nomeInstrumentoPluv
        data.append(go.Bar(x=df_.index,y=df_[y_bar], name='Pluviometria', marker=dict(color='#291BEE',line=dict(width=0))))
        
    y_line = nomeInstrumento
    df = df_.copy(deep=True)
    if ymax_outlier!=None:
        df = df[df[nomeInstrumento]<=ymax_outlier]
    if ymin_outlier!=None:
        df = df[df[nomeInstrumento]>=ymin_outlier]
    
    leiturasInstrumentos = df[nomeInstrumento].copy(deep=True)
    leiturasInstrumentos = leiturasInstrumentos

    try:
        print(leiturasInstrumentos.columns)       
    except:
        data.append(go.Scatter(x=leiturasInstrumentos.index,y=leiturasInstrumentos.values, mode='lines', name=nomeInstrumento, line=dict(color='red'), yaxis="y2",connectgaps=True))
    else:
        data.append(go.Scatter(x=leiturasInstrumentos.index,y=leiturasInstrumentos[y_line], mode='lines', name=nomeInstrumento, line=dict(color='red'), yaxis="y2",connectgaps=True))
    

    
    y_scatter = f"{nomeInstrumento} (seco)"
    try:
        data.append(go.Scatter(x=df.index,y=df[y_scatter], mode='markers', name=y_scatter, marker=dict(color='red',symbol='x'), yaxis="y2"))
    except Exception as m:
        print(f"Exception: {m}")
    
    if temCotaFundo:
        y2_line = f"{nomeInstrumento} (cota de fundo)"
        data.append(go.Scatter(x=df.index,y=df[y2_line], mode='lines', name="Cota de fundo", line=dict(color='black',dash="dash"), yaxis="y2",connectgaps=True))
    
    if temCotaTopo:
        y3_line = f"{nomeInstrumento} (cota de topo)"
        data.append(go.Scatter(x=df.index,y=df[y3_line], mode='lines', name="Cota de topo", line=dict(color='black',dash="dash"), yaxis="y2",connectgaps=True))
    
    if temNVControle:
        y4_line = f"{nomeInstrumento} (atenção)"
        y5_line = f"{nomeInstrumento} (alerta)"
        y6_line = f"{nomeInstrumento} (emergência)"
        data.append(go.Scatter(x=df.index,y=df[y4_line], mode='lines', name="Atenção", line=dict(color='yellow',dash="dash"), yaxis="y2",connectgaps=True))
        data.append(go.Scatter(x=df.index,y=df[y5_line], mode='lines', name="Alerta", line=dict(color='orange',dash="dash"), yaxis="y2",connectgaps=True))
        data.append(go.Scatter(x=df.index,y=df[y6_line], mode='lines', name="Emergência", line=dict(color='red',dash="dash"), yaxis="y2",connectgaps=True))



    y2inf,y2sup,dV = intervaloPerfeito(nomeInstrumento)
    if intervaloY != []:
        y2inf,y2sup,dV = intervaloY
    # print(f"Intervalo perfeito de {nomeInstrumento}: {y2inf}, {y2sup}, {dV}")
    

    # Criar a figura e adicionar as traces
    fig:go.Figure = go.Figure(data=data)

    # Configurar layout do gráfico
    novoLayout = dict(
        # autosize=True,
        width=900,
        height=400,
        title=dict(
            text=f'Histórico do {nomeInstrumento}',
            font=dict(
                size=12,
                weight=70
            ),
            # xref="paper",
            xanchor="left",
            yanchor="top",
            pad=dict(
                b=0,
                t=0
            )        
        ),
        font=dict(
            family="Arial",
            color="black",
            weight=40
        ),
        margin=dict(
            autoexpand=True,
            b=50,
            t=20,
            l=40,
            r=0
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.0,
            xanchor="right",
            x=1,
            bgcolor='rgba(255, 255, 255, 0.5)',  # Cor de fundo da legenda
            bordercolor='black',  # Cor da borda da legenda
            borderwidth=1  # Largura da borda da legenda
        ),
        xaxis=dict(
            # title='Data de Medição',
            tickformat='%b/%Y', # mês abreviado, contudo aparece em inglês
            # tickformat='%d/%m/%Y',
            showgrid=True,
            gridcolor="#acadbc",
            linecolor="#acadbc",
            zeroline=False,
            range=[dataInicio,dataFim],
            dtick="M6",
            tickangle=-90
        ),
        yaxis=dict(
            title=dict(
                text='Pluviometria (mm)',
                standoff=10
            ),
            linecolor="#acadbc",
            showgrid=False,
            range=[pluvLimInf,pluvLimSup],
            dtick=dV_pluv,
            zeroline=True
        ),
        yaxis2=dict(
            title=dict(
                text=nomeEixo,
                standoff=10
            ),
            overlaying='y',
            showgrid=True,
            gridcolor="#acadbc",
            linecolor="#acadbc",
            range=[y2inf,y2sup],
            dtick=dV,
            side="right"
        ),
        barmode='group',  # Opcional: agrupar barras
        plot_bgcolor='#F5F5FA',  # Cor de fundo do gráfico
        paper_bgcolor='white'  # Cor de fundo do papel do gráfico
    )
    fig.update_layout(novoLayout)

    import os

    if not os.path.exists(f"images"):
        os.mkdir(f"images")
    if not os.path.exists(f"images/{estrutura}"):
        os.mkdir(f"images/{estrutura}")
    if not os.path.exists(f"images/{estrutura}/{tipo}"):
        os.mkdir(f"images/{estrutura}/{tipo}")

    fig.write_html(f"images/{estrutura}/{tipo}/{nomeInstrumento}.html")
    # AVALIAR A POSSIBILIDADE DE SALVAR IMAGEM DINÂMICA COM ORCA OU KALEIDO (NÃO ESTÁ FUNCIONANDO)
    # https://plotly.com/python/static-image-export/

In [48]:
def main():
    estrutura = "Monjolo"
    tipoInstrumento = "Piezômetro" # Colocar uma string vazia para fazer com todos
    if estrutura in estruturasInteresse or estrutura=="":
        print(f"Os gráficos da estrutura {estrutura} começaram a ser gerados.")
        if estrutura=="":
            estruturas = estruturasInteresse
        else:
            estruturas = [estrutura]
        for estrutura in estruturas:
            for nomeInstrumento in listarInstrumentos(estrutura=estrutura,tipoInstrumento=tipoInstrumento):
                print(get_info_instrument(nameInstrument=nomeInstrumento,nameInfo=colunaEstrutura),nomeInstrumento,len(leituras[leituras[colunaCodigo]==nomeInstrumento]))
                
                if nomeInstrumento in list(leituras[colunaCodigo]):
                    
                    gerarGrafico(nomeInstrumento=nomeInstrumento,
                        temPLV=True, # Com pluviometria
                        temNVControle=False,
                        temCotaFundo=False,
                        temCotaTopo=False
                        )
        print("Os gráficos foram finalizados.")
    else:
        raise Exception(f"A estrutura {estrutura} não faz parte do escopo de interesse.")


# Comando para rodar todos os instrumentos da estrutura e tipo
# main()


# GRÁFICOS gerados a mão
# GRÁFICOS COM OUTLIERS NÃO DECLARADOS
# INTERVALOS FEITOS NA MÃO
# YMAX_OUTLIER É O VALOR QUE SIGNIFICA O MAIOR VALOR PERMITIDO PARA O HISTÓRICO

# gerarGrafico(nomeInstrumento="AGLBRMMI002_A", intervaloY=[0,80,10], ymax_outlier=500.0)
# gerarGrafico(nomeInstrumento="AGLBRMMI002", intervaloY=[0,3,.5], ymax_outlier=50.0)
# gerarGrafico(nomeInstrumento="AGLBRMRR002_A", intervaloY=[657.0,658.0,.1], ymax_outlier=658.)

# INSTRUMENTOS JORRANTES (POR HORA FOI RETIRADA AS LEITURAS JORRANTES)
# gerarGrafico(nomeInstrumento="AGLBRMPZ001")
# gerarGrafico(nomeInstrumento="AGLBRMPZ001_A",ymin_outlier=300,intervaloY=[639,649,1])
# gerarGrafico(nomeInstrumento="AGLBRMPZ004")
# gerarGrafico(nomeInstrumento="AGLBRMPZ004_A",ymin_outlier=300,intervaloY=[639.5,644,.5])
# gerarGrafico(nomeInstrumento="AGLBRMPZ005")
# gerarGrafico(nomeInstrumento="AGLBRMPZ005_A",ymin_outlier=300,intervaloY=[639,644,.5],ymax_outlier=646)

# Instrumentos com erro nos eixos
# gerarGrafico(nomeInstrumento="AGLBDIGMO001",intervaloY=[0,1000,100])
# gerarGrafico(nomeInstrumento="AGLBDIGMT001_A",ymax_outlier=1000.,intervaloY=[0,1000,100])
# gerarGrafico(nomeInstrumento="AGLBDIGPZ009_A",ymin_outlier=1.,intervaloY=[634,638,.5])




import winsound
# Documention: https://docs.python.org/pt-br/3.7/library/winsound.html
# Frequências das notas musicais: https://iazzetta.eca.usp.br/tutor/acustica/introducao/tabela1.html
winsound.Beep(293, 400) # Tocar beep na nota D3 por 4 segundos (4000ms)

In [43]:
def avaliarTrecho (nomeInstrumento,y0=0,yi=10**10,percentiles=[.25,.5,.75]):
    serie:pd.date_range = tratarDados(nomeInstrumento)
    serie = serie[(y0<=serie[nomeInstrumento]) & (serie[nomeInstrumento]<=yi)]
    return serie.describe(percentiles=percentiles)
avaliarTrecho("AGLBDIGMO001",percentiles=[.90,.95,.98,.99])

Código do Instrumento,AGLBDIGMO001
count,93.000000
mean,216.791129
std,120.166816
min,39.700000
50%,172.100000
90%,318.651111
95%,421.742222
98%,629.752000
99%,671.512000
max,840.700000


In [21]:
# leituras[leituras[colunaCodigo]=="AGLBRMPZ001"][leituras["Condição Adversa"]=="JORRANTE"]

# instrumentosSecoes = ["AGLBRMMI002",
# "AGLBRMRR002_A"]
# data = []
# for i in instrumentosSecoes:
#     dfInstrumento = tratarDados(i)
#     data.append(go.Scatter(x=dfInstrumento.index,y=dfInstrumento[i],name=i,mode="lines"))
#     try:
#         dfInstrumento = tratarDados(i,seco=True)
#         i = f"{i} (seco)"
#         data.append(go.Scatter(x=dfInstrumento.index,y=dfInstrumento[i],name=i,mode="markers"))
#     except Exception as m:
#         print(m)
# fig = go.Figure(data=data)
# fig.update_layout(
#     dict1=dict(
#         xaxis=dict(
#             # title='Data de Medição',
#             # tickformat='%b/%Y', # mês abreviado, contudo aparece em inglês
#             tickformat='%d/%m/%Y',
#             showgrid=True,
#             # gridcolor="#acadbc",
#             linecolor="#acadbc",
#             zeroline=False,
#             # range=[dataInicio,dataFim],
#             dtick="M6",
#             tickangle=-90
#             )
#         )
#     )

TypeError: isEvery() missing 2 required positional arguments: 'iterable' and 'type_'